In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot

In [2]:
pd.options.display.max_rows = 50
pd.options.display.max_columns = 200

In [3]:
#list columns to be included
include = ['CaseID', 'SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'PRNCPTX', 'CPT', 'WORKRVU', 'Age', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'SEPSHOCKPATOS', 'OTHERCPT1', 'CONCPT1', 'OTHERCPT2', 'CONCPT2', 'INOUT', 'TRANST','OperYR', 'OPTIME', 'HtoODay', 'SSSIPATOS', 'DSSIPATOS', 'OSSIPATOS', 'PNAPATOS', 'VENTPATOS', 'UTIPATOS', 'SEPSISPATOS', 'SEPSHOCKPATOS', 'OPTIME','SUPINFEC', 'WNDINFD', 'ORGSPCSSI']

CPTs = ['OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10',]

col_features = ['COL_STEROID', 'COL_MECH_BOWEL_PREP', 'COL_ORAL_ANTIBIOTIC', 'COL_CHEMO', 'COL_INDICATION', 'COL_ICD9_INDICATION', 'COL_EMERGENT', 'COL_ICD9_EMERGENT', 'COL_APPROACH', 'COL_ICD10_INDICATION', 'COL_ICD10_EMERGENT']
incl = list(map(lambda x:x.upper(), include))
total = col_features + incl + CPTs

In [4]:
#make sure CPT's are read in as strings
CPT_dict = {'OTHERCPT1': str, 'OTHERCPT2':str, 'OTHERCPT3':str, 'OTHERCPT4':str, 'OTHERCPT5':str, 'OTHERCPT6':str, 'OTHERCPT7':str, 'OTHERCPT8':str, 'OTHERCPT9':str, 'OTHERCPT10':str, 'CONCPT1':str, 'CONCPT2':str,'CONCPT3':str,'CONCPT4':str,'CONCPT5':str,'CONCPT6':str,'CONCPT7':str,'CONCPT8':str,'CONCPT9':str,'CONCPT10':str}

In [5]:
#read in the data from raw csv
data = pd.read_csv(r'/home/kchen/Documents/nsqip_raw/procol/procol.csv', index_col='CASEID', usecols=total, low_memory=False, dtype=CPT_dict, na_values=[-99, '-99', 'Unknown'])

In [6]:
data.shape

(276232, 93)

In [10]:
#create a variable called 'miss' that indicates if primary outcomes are missing
data['miss'] = ((data['SUPINFEC'].isnull()) | (data['WNDINFD'].isnull()) | (data['ORGSPCSSI'].isnull()))

In [11]:
data['miss'].value_counts()

False    275152
True       1080
Name: miss, dtype: int64

In [15]:
#drop rows with missing primary outcomes
data.drop(data[data['miss'] == True].index, inplace = True)

In [19]:
#check they have been dropped
data['miss'].value_counts()

False    275152
Name: miss, dtype: int64

In [16]:
data['SUPINFEC'].value_counts(normalize=True)

No Complication               0.955581
Superficial Incisional SSI    0.044419
Name: SUPINFEC, dtype: float64

In [17]:
data['WNDINFD'].value_counts(normalize=True)

No Complication        0.991052
Deep Incisional SSI    0.008948
Name: WNDINFD, dtype: float64

In [18]:
data['ORGSPCSSI'].value_counts(normalize=True)

No Complication    0.940953
Organ/Space SSI    0.059047
Name: ORGSPCSSI, dtype: float64

In [21]:
#create the primary outcome 'ssi', which is superficial, deep, and organ-space SSI combined
data['ssi'] = ((data['SUPINFEC'] == 'Superficial Incisional SSI') | (data['WNDINFD'] == 'Deep Incisional SSI') | (data['ORGSPCSSI'] == 'Organ/Space SSI'))

In [22]:
data['ssi'].value_counts(normalize=True, dropna=False)

False    0.893226
True     0.106774
Name: ssi, dtype: float64

In [19]:
#recode 'SUPINFEC' to 0 for SSI and 1 for non-SSI
data['SUPINFEC'] = data['SUPINFEC'].replace(['Superficial Incisional SSI', 'No Complication'], [1, 0])
#recode 'WNDINFD' to 0 for SSI and 1 for non-SSI
data['WNDINFD'] = data['WNDINFD'].replace(['Deep Incisional SSI', 'No Complication'], [1, 0])
#recode 'ORGSPCSSI' to 0 for SSI and 1 for non-SSI
data['ORGSPCSSI'] = data['ORGSPCSSI'].replace(['Organ/Space SSI', 'No Complication'], [1, 0])

0    262930
1     12222
Name: SUPINFEC, dtype: int64

In [25]:
#Create BMI column
lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)


In [26]:
#BMI <12 set to 12, BMI >50 set to 50
data.loc[data['BMI'] > 60, 'BMI'] = 60
data.loc[data['BMI'] < 12, 'BMI'] = 12


In [27]:
#Convert 90+ to 91
data.loc[data['AGE'] == '90+', 'AGE'] = 91

CASEID
6629599     44.0
6629786     38.0
6630805     84.0
6630866     71.0
6631518     50.0
            ... 
10363817    50.0
10363850    53.0
10363959    65.0
10363964    41.0
10363966    72.0
Name: AGE, Length: 275152, dtype: float64

In [28]:
#list CPT codes including ostomy placement and create column 'ost' marking ostomy
data['ost'] = 0
ost = [44211, 44212, 45113, 45119, 44155, 44157, 44158, 44125, 44187, 44141, 44143, 44144, 44146, 44150, 44151, 44206, 44208, 44210, 44187, 44188, 44320, 44310]
for CPT in ost:
    data.loc[data['CPT'] == CPT, 'ost'] = 1
data['ost'].value_counts(dropna=False)
#create a list of CPT's as strings and search other+concurrent CPT for those
str_ost = []
for x in ost:  
    str_ost.append(str(x))
for i in range(1,11):
    for CPT in str_ost:
        data.loc[data['OTHERCPT%s' % i].str.contains(CPT, na=False), 'ost'] = 1
        data.loc[data['CONCPT%s' % i].str.contains(CPT, na=False), 'ost'] = 1
data['ost'].value_counts(dropna=False)

0    202782
1     72370
Name: ost, dtype: int64

In [29]:
#condense 'COL_APPROACH' column categories
data.loc[data['COL_APPROACH'] == 'SILS', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'SILS w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'SILS w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Endoscopic', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Endoscopic w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Endoscopic w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'NOTES', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'NOTES w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'NOTES w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Other MIS approach', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Other MIS approach w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Other MIS approach w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Hybrid', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Hybrid w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Hybrid w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Robotic', 'COL_APPROACH'] = 'Robotic'
data.loc[data['COL_APPROACH'] == 'Robotic w/ open assist', 'COL_APPROACH'] = 'Robotic'
data.loc[data['COL_APPROACH'] == 'Robotic w/ unplanned conversion to open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Laparoscopic w/ open assist', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Laparoscopic Hand Assisted', 'COL_APPROACH'] = 'Laparoscopic'
data.loc[data['COL_APPROACH'] == 'Laparoscopic w/ unplanned conversion to Open', 'COL_APPROACH'] = 'Laparoscopic w/ unplanned conversion to open'

data.loc[data['COL_APPROACH'] == 'Open', 'COL_APPROACH'] = 'Open (planned)'

data['COL_APPROACH'].value_counts()

Laparoscopic                                    130393
Open (planned)                                  100059
Robotic                                          23864
Laparoscopic w/ unplanned conversion to open     20344
Other                                              405
Name: COL_APPROACH, dtype: int64

In [30]:
data['COMBCPT'] = 0

In [31]:
#condense 'CPT' categories to fewer/simpler categories
data.loc[data['CPT'] == 44204, 'COMBCPT'] = 'Laparoscopic partial colectomy'
data.loc[data['CPT'] == 44207, 'COMBCPT'] = 'Laparoscopic L colectomy'
data.loc[data['CPT'] == 44208, 'COMBCPT'] = 'Laparoscopic L colectomy'
data.loc[data['CPT'] == 44206, 'COMBCPT'] = 'Laparoscopic L colectomy'
data.loc[data['CPT'] == 44205, 'COMBCPT'] = 'Laparoscopic R colectomy'
data.loc[data['CPT'] == 44140, 'COMBCPT'] = 'Partial colectomy'
data.loc[data['CPT'] == 44144, 'COMBCPT'] = 'Partial colectomy'
data.loc[data['CPT'] == 44141, 'COMBCPT'] = 'Partial colectomy'
data.loc[data['CPT'] == 44160, 'COMBCPT'] = 'R colectomy'
data.loc[data['CPT'] == 44145, 'COMBCPT'] = 'L colectomy'
data.loc[data['CPT'] == 44143, 'COMBCPT'] = 'L colectomy'
data.loc[data['CPT'] == 44146, 'COMBCPT'] = 'L colectomy'
data.loc[data['CPT'] == 44146, 'COMBCPT'] = 'L colectomy'
data.loc[data['CPT'] == 44210, 'COMBCPT'] = 'Laparoscopic total colectomy'
data.loc[data['CPT'] == 44150, 'COMBCPT'] = 'Total colectomy'
data.loc[data['CPT'] == 44151, 'COMBCPT'] = 'Total colectomy'
data.loc[data['CPT'] == 44156, 'COMBCPT'] = 'Total colectomy'
data.loc[data['CPT'] == 44157, 'COMBCPT'] = 'Total colectomy'
data.loc[data['CPT'] == 44158, 'COMBCPT'] = 'Total colectomy'
data.loc[data['CPT'] == 45110, 'COMBCPT'] = 'APR'
data.loc[data['CPT'] == 45112, 'COMBCPT'] = 'APR'
data.loc[data['CPT'] == 45395, 'COMBCPT'] = 'Laparoscopic APR'
data.loc[data['CPT'] == 45130, 'COMBCPT'] = 'Proctectomy, perineal approach'
data.loc[data['CPT'] == 45123, 'COMBCPT'] = 'Proctectomy, perineal approach'
data.loc[data['CPT'] == 45135, 'COMBCPT'] = 'Proctectomy, perineal approach'
data.loc[data['CPT'] == 45121, 'COMBCPT'] = 'Proctectomy, perineal approach'
data.loc[data['CPT'] == 45395, 'COMBCPT'] = 'Laparoscopic LAR'
data.loc[data['CPT'] == 45111, 'COMBCPT'] = 'LAR'
data.loc[data['CPT'] == 45113, 'COMBCPT'] = 'LAR'
data.loc[data['CPT'] == 45119, 'COMBCPT'] = 'LAR'
data.loc[data['CPT'] == 45120, 'COMBCPT'] = 'LAR'
data.loc[data['CPT'] == 45397, 'COMBCPT'] = 'LAR'

data.loc[data['CPT'] == 44212, 'COMBCPT'] = 'Laparoscopic proctocolectomy'
data.loc[data['CPT'] == 44211, 'COMBCPT'] = 'Laparoscopic proctocolectomy'
data.loc[data['CPT'] == 44155, 'COMBCPT'] = 'Proctocolectomy'
data.loc[data['CPT'] == 44156, 'COMBCPT'] = 'Proctocolectomy'


data.loc[data['CPT'] == 45126, 'COMBCPT'] = 'Pelvic exenteration'

data.loc[data['CPT'] == 44147, 'COMBCPT'] = 'Colectomy, combined transanal approach'
data.loc[data['CPT'] == 45550, 'COMBCPT'] = 'Laparoscopic rectopexy'
data.loc[data['CPT'] == 45402, 'COMBCPT'] = 'Laparoscopic rectopexy'

data.loc[data['CPT'] == 45114, 'COMBCPT'] = 'Proctectomy, transsacral approach'
data.loc[data['CPT'] == 45160, 'COMBCPT'] = 'Proctectomy, transsacral approach'
data.loc[data['CPT'] == 45116, 'COMBCPT'] = 'Proctectomy, transsacral approach'


In [32]:
data['COMBCPT'].value_counts()

Laparoscopic partial colectomy            52111
Laparoscopic L colectomy                  47321
Partial colectomy                         41307
L colectomy                               40108
Laparoscopic R colectomy                  35328
R colectomy                               26410
Total colectomy                            7596
Laparoscopic total colectomy               6850
LAR                                        4432
APR                                        3847
Laparoscopic LAR                           2977
Proctectomy, perineal approach             2201
Laparoscopic proctocolectomy               2183
Proctocolectomy                             963
Laparoscopic rectopexy                      565
Pelvic exenteration                         446
Colectomy, combined transanal approach      392
Proctectomy, transsacral approach           115
Name: COMBCPT, dtype: int64

In [33]:
#generate a new column describing race and ethnicity
data.groupby('RACE_NEW')['ETHNICITY_HISPANIC'].value_counts()

RACE_NEW                             ETHNICITY_HISPANIC
American Indian or Alaska Native     No                      1029
                                     Yes                       92
Asian                                No                      8158
                                     Yes                       53
Black or African American            No                     23525
                                     Yes                      234
Native Hawaiian or Pacific Islander  No                       551
                                     Yes                       32
Unknown/Not Reported                 No                      5116
                                     Yes                     4394
White                                No                    190646
                                     Yes                     8937
Name: ETHNICITY_HISPANIC, dtype: int64

In [34]:
data['RACE'] = data['RACE_NEW']

In [35]:
data.loc[data['ETHNICITY_HISPANIC'] == 'Yes', 'RACE'] = 'Hispanic'
data['RACE'].value_counts(dropna=False)

White                                  194329
Unknown/Not Reported                    32965
Black or African American               24277
Hispanic                                13742
Asian                                    8244
American Indian or Alaska Native         1039
Native Hawaiian or Pacific Islander       556
Name: RACE, dtype: int64

In [36]:
data['RACE'] = data['RACE'].fillna('Unknown/Not Reported')

#Replace 'Unknown/Not Reported' with 'Unknown'

data['RACE'] = data['RACE'].replace('Unknown/Not Reported', 'Unknown')
data['RACE'].value_counts(dropna=False)

White                                  194329
Unknown                                 32965
Black or African American               24277
Hispanic                                13742
Asian                                    8244
American Indian or Alaska Native         1039
Native Hawaiian or Pacific Islander       556
Name: RACE, dtype: int64

In [37]:
#drop the separate race and ethnicity columns
data.drop(['RACE_NEW', 'ETHNICITY_HISPANIC'], axis=1, inplace=True)

In [38]:
data['COL_INDICATION'].value_counts()

Colon cancer                         90117
Chronic diverticular disease         36070
Other-Enter ICD-10 for diagnosis     29838
Acute diverticulitis                 21681
Non-malignant polyp                  20076
Other-Enter ICD-9 for diagnosis      16393
Crohn's Disease                      15997
Colon cancer w/ obstruction          11870
Ulcerative colitis                    8679
Rectal cancer                         8571
Volvulus                              7098
Rectal prolapse                       2472
Bleeding                              1788
Enterocolitis (e.g. C. Difficile)     1463
Anal cancer                            420
Name: COL_INDICATION, dtype: int64

In [39]:
#add an additional indication from the ICD9 codes
data.loc[data['COL_ICD9_INDICATION'] == '211.3', 'COL_INDICATION'] = 'Non-malignant polyp'
data['COL_INDICATION'].value_counts()

Colon cancer                         90114
Chronic diverticular disease         36070
Other-Enter ICD-10 for diagnosis     29838
Acute diverticulitis                 21681
Non-malignant polyp                  20799
Crohn's Disease                      15997
Other-Enter ICD-9 for diagnosis      15674
Colon cancer w/ obstruction          11869
Ulcerative colitis                    8679
Rectal cancer                         8571
Volvulus                              7098
Rectal prolapse                       2472
Bleeding                              1788
Enterocolitis (e.g. C. Difficile)     1463
Anal cancer                            420
Name: COL_INDICATION, dtype: int64

In [40]:
#save out the dataframe to csv for use in the table1 script
data.to_csv(r'/media/kchen/2TB/kchen_backup/ssi/data/table1ssi.csv')

In [41]:
#drop the missing column
outcomes = ['miss']
data.drop(outcomes, axis=1, inplace=True)

In [42]:
#define the numerical and categorical columns
num_cols = ['AGE','HEIGHT','WEIGHT','BMI','PRSODM','PRBUN','PRCREAT','PRALBUM','PRBILI','PRSGOT','PRALKPH','PRWBC','PRHCT','PRPLATE','PRPTT','PRINR','PRPT','OPTIME', 'HTOODAY','WORKRVU']
cat_cols = list(set(total) - set(num_cols) - set(outcomes) - set(['CASEID','ETHNICITY_HISPANIC','RACE_NEW']))
cat_cols.append('RACE')

In [43]:
#create a copy to work with
data_imputed = data.copy()

In [44]:
#drop the other and concurrent CPT columns
drop_CPT = ['OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10',]
data_imputed.drop(drop_CPT, axis=1, inplace=True)

In [46]:
cat_cols = list(set(cat_cols) - set(drop_CPT))

In [47]:
#fill in missing categorical values with 'Unknown'
for col in cat_cols:
    data_imputed[col].fillna(value='Unknown', inplace=True)

In [48]:
#list numerical columns by percentage of missing data
col_missing = data_imputed.isnull().sum()/data.shape[0]
col_missing = col_missing.sort_values(ascending=False)
col_missing = col_missing[col_missing > 0]
(pd.DataFrame(col_missing)).head(10)

,0
PRPT,0.992833
PRPTT,0.639865
PRINR,0.507698
PRSGOT,0.297712
PRALKPH,0.278163
PRBILI,0.277621
PRALBUM,0.275869
PRBUN,0.099174
PRSODM,0.067505
PRCREAT,0.061239


In [49]:
#drop columns with >25% missing data
drop = ['PRPT', 'PRPTT','PRSGOT', 'PRALKPH','PRBILI','COMBCPT']
data_imputed.drop(drop, axis=1, inplace=True)

In [50]:
num_cols = list(set(num_cols) - set(drop))

In [51]:
#impute missing numerical columns with median
num_imputer = SimpleImputer(strategy = 'median')

# Imputation for numeric:
for ncol in num_cols:
    data_imputed[ncol] = num_imputer.fit_transform(data_imputed[ncol].values.reshape(-1, 1))

In [52]:
#scale the data with RobustScaler
for col in num_cols:
    data_imputed[col] = RobustScaler().fit_transform(data_imputed[col].values.reshape(-1,1))

In [53]:
#label encode the categorical columns
le=LabelEncoder()
for col in cat_cols:
        data_imputed[col] = le.fit_transform(data_imputed[col].astype(str))

In [54]:
#recode the primary outcome to 0/1
data_imputed.loc[data_imputed['ssi'] == True, 'ssi'] = 1
data_imputed.loc[data_imputed['ssi'] == False, 'ssi'] = 0

In [55]:
data['ssi'].value_counts()

False    245773
True      29379
Name: ssi, dtype: int64

In [56]:
#separate out the 2019 data and save it to a separate csv ('test')
data19 = data_imputed[data_imputed['OPERYR'] == 7]
data19.shape

(51250, 72)

In [57]:
drop19 = data_imputed[data_imputed['OPERYR'] != 7]
drop19.shape

(223902, 72)

In [58]:
data19 = data19.drop(['OPERYR'], axis=1)
drop19 = drop19.drop(['OPERYR'], axis=1)

In [59]:
data19.to_csv(r'/media/kchen/2TB/kchen_backup/ssi/data/procol_test.csv')
drop19.to_csv(r'/media/kchen/2TB/kchen_backup/ssi/data/procol_train.csv')

In [60]:
import feather
data19.reset_index(inplace=True)
drop19.reset_index(inplace=True)
data19 = data19.drop(['CASEID'], axis=1)
drop19 = drop19.drop(['CASEID'], axis=1)

In [61]:
data19.to_feather(r'/media/kchen/2TB/kchen_backup/ssi/data/procol_test.feather')
drop19.to_feather(r'/media/kchen/2TB/kchen_backup/ssi/data/procol_train.feather')